In [1]:
%pip install pandas
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os

In [3]:
df = pd.read_parquet("data/Thermo_SRM_Pool_meta_data.parquet", engine='fastparquet')

In [4]:
print(df.shape)
print(df.head(4))

(6080606, 15)
                                       raw_file  scan_number  \
0  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34736   
1  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        25591   
2  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        24831   
3  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34870   

      modified_sequence  precursor_charge  precursor_intensity          mz  \
0  LPGSLETYVEQEQGENANDR                 2           29525630.0  1125.01950   
1       HGSLQEYLQNDTGSK                 2           13188580.0   838.89737   
2        VEEEEEINSELTAR                 2           20663460.0   824.38904   
3  LPGSLETYVEQEQGENANDR                 2           19884630.0  1125.01950   

   precursor_mz fragmentation mass_analyzer  retention_time  \
0   1125.018331           CID          ITMS          38.411   
1    838.894401           HCD          FTMS          30.394   
2    824.388401           CID          ITMS          29.737   
3   1125.018831       

In [5]:
print(df.columns)

Index(['raw_file', 'scan_number', 'modified_sequence', 'precursor_charge',
       'precursor_intensity', 'mz', 'precursor_mz', 'fragmentation',
       'mass_analyzer', 'retention_time', 'indexed_retention_time',
       'andromeda_score', 'peptide_length', 'orig_collision_energy',
       'aligned_collision_energy'],
      dtype='object')


Needed: modified_sequence, precursor charge, precursor_intensity (abundance), mz

In [6]:
for index, seq in enumerate(df["modified_sequence"]):
    if seq == "LPGSLETYVEQEQGENANDR":
        print(index, df["precursor_charge"][index], df["scan_number"][index])
        # print df row
        print("---"+ df.iloc[index].to_string())
        print("      ------------------------")

0 2 34736
---raw_file                    01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2
scan_number                                                        34736
modified_sequence                                   LPGSLETYVEQEQGENANDR
precursor_charge                                                       2
precursor_intensity                                           29525630.0
mz                                                             1125.0195
precursor_mz                                                 1125.018331
fragmentation                                                        CID
mass_analyzer                                                       ITMS
retention_time                                                    38.411
indexed_retention_time                                         62.827337
andromeda_score                                                   447.14
peptide_length                                                        20
orig_collision_energy                 